In [1]:
import numpy as np

In [2]:
prior = np.array([1/3., 1/3, 1/3])
prior

array([0.33333333, 0.33333333, 0.33333333])

In [3]:
likelihood = np.array([.2, .4, .7])
likelihood

array([0.2, 0.4, 0.7])

In [8]:
posterior_unnormalized = prior * likelihood
posterior_unnormalized

array([0.06666667, 0.13333333, 0.23333333])

In [9]:
Z = posterior_unnormalized.sum() # normalization constant

In [10]:
# [P(A|H), P(B|H), P(C|H)]
posterior = posterior_unnormalized / Z
posterior

array([0.15384615, 0.30769231, 0.53846154])

In [11]:
prior = np.array([.7, .2, .1])
prior

array([0.7, 0.2, 0.1])

In [12]:
likelihood = np.array([.2, .4, .7])
likelihood

array([0.2, 0.4, 0.7])

In [13]:
posterior_unnormalized = prior * likelihood
posterior_unnormalized

array([0.14, 0.08, 0.07])

In [14]:
Z = posterior_unnormalized.sum() # normalization constant

In [15]:
# [P(A|H), P(B|H), P(C|H)]
posterior = posterior_unnormalized / Z
posterior

array([0.48275862, 0.27586207, 0.24137931])

---

In [11]:
prior = np.array([.7, .])
prior

array([0.7, 0.2, 0.1])

In [12]:
likelihood = np.array([.2, .4, .7])
likelihood

array([0.2, 0.4, 0.7])

In [13]:
posterior_unnormalized = prior * likelihood
posterior_unnormalized

array([0.14, 0.08, 0.07])

In [14]:
Z = posterior_unnormalized.sum() # normalization constant

In [15]:
# [P(A|H), P(B|H), P(C|H)]
posterior = posterior_unnormalized / Z
posterior

array([0.48275862, 0.27586207, 0.24137931])

---

In [17]:
df = np.array([4,6,8,12,20])
df

array([ 4,  6,  8, 12, 20])

In [18]:
prior = np.array([1/5]*5)
prior

array([0.2, 0.2, 0.2, 0.2, 0.2])

In [27]:
obs = 6
like = []
for d in df:
    if d < obs:
        like.append(0)
    else:
        like.append(1./d)
    print(f'{d}side:  prob=', 1/d)
like = np.array(like);
like

4side:  prob= 0.25
6side:  prob= 0.16666666666666666
8side:  prob= 0.125
12side:  prob= 0.08333333333333333
20side:  prob= 0.05


array([0.        , 0.16666667, 0.125     , 0.08333333, 0.05      ])

In [25]:
post = prior * like
Z = post.sum()
post = post / Z
post

array([0.        , 0.39215686, 0.29411765, 0.19607843, 0.11764706])

---

# Bertrand's box paradox
- https://en.wikipedia.org/wiki/Bertrand%27s_box_paradox
- Statistical Rethinking 2nd, Chapter 15. Missing Data

Suppose a robot cooks three pancakes. The first pancake is burnt on both sides (BB). The second pancake is burnt on only on side (BU). The third pancake is not burnt at all (UU). Now another robot is serving you at random one of these pancakes, and the side facing up on your plate is burnt. What is the probability that the other side is  also burnt?

In [144]:
faceup = 0    
facedown = 1
B = 1
U = 0
pancakes = np.array( [ [B,B], [B,U], [U,U]] )

In [89]:
n_repeat = 1000000
burntfaceup = []
for _ in range(n_repeat):
    i = np.random.randint(low=0, high=3)
    choice = pancakes[i]
    if choice[faceup] == B: # observation
        burntfaceup.append(choice)

In [90]:
len_total = len(burntfaceup)

In [91]:
burntdown = []
for pc in burntfaceup:
    if pc[facedown] == B:
        burntdown.append(pc)

In [92]:
len(burntdown) / len_total

0.5001769624839534

- Something wrong
- when the pancake is put on a plate, either side can be up.

In [138]:
n_repeat = 1000000
burntfaceup = []
for k in range(n_repeat):
    i = np.random.randint(low=0, high=3)
    choice = pancakes[i]  # the robot chooses this pancake
    # up-side down
    if np.random.uniform() < 0.5:  # 50% chance
        choice[0], choice[1] = choice[1], choice[0]
    if choice[faceup] == B: # observation
        burntfaceup.append(choice)

In [139]:
len_total = len(burntfaceup)

In [140]:
burntdown = []
for pc in burntfaceup:
    if pc[facedown] == B:
        burntdown.append(pc)

In [141]:
len(burntdown) / len_total

0.6659746306807954

## let's make it faster with numpy vectorization

In [213]:
n_repeat = 10000000
i = np.random.randint(0,3, size=n_repeat);
choice = pancakes[i];
# up-side down
u = np.random.uniform(size=n_repeat)
do_swap = u < 0.5; do_swap;
a = choice[do_swap == False]
b = choice[do_swap == True]
b[:,[0,1]] = b[:,[1,0]]  # swap == upside down
choice = np.vstack((a,b))           # random generation
#
choice = choice[ choice[:,faceup] == B]  # observation
#
total_len = len(choice)
burnt_down = choice[:,facedown] == B       # target of interest
#
burnt_down.sum() / total_len        # the ratio

0.6667841501635007